# Computational Theory

## Introduction

## Imports

In [17]:
import numpy as np

## Problem 1 - Binary Words & Operations

### Implementing Parity

In [18]:
def parity(x, y, z):
    """
    Calculate the parity (bitwise XOR) of 3 32-bit unsigned integers.

    """

    return x ^ y ^ z

#### Test Cases

In [19]:
assert parity(1, 0, 0) == np.uint32(1)
assert parity(1, 1, 0) == np.uint32(0)
assert parity(1, 1, 1) == np.uint32(1)

print("All test cases pass")

All test cases pass


## Problem 2 - Fractional Parts of Cube Roots

## Problem 3 - Padding

## Problem 4 - Hashes

## Problem 5 - Passwords

## Conclusion

## End